In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

###CREATE FLAG PARAMETER

In [0]:
dbutils.widgets.text('incremental_flag','0')

In [0]:
incremental_flag=dbutils.widgets.get('incremental_flag')

In [0]:
print(incremental_flag)

#Creating Dimension Model

In [0]:
df_src =spark.sql('''
                  SELECT * 
                  FROM parquet.`abfss://silver@carkrishdatalake.dfs.core.windows.net/Carsales`
                  ''')

In [0]:
df_src.display()

In [0]:
df_src=spark.sql('''
                 SELECT DISTINCT (Dealer_Id) as Dealer_Id, DealerName
                 FROM parquet.`abfss://silver@carkrishdatalake.dfs.core.windows.net/Carsales`
                 ''')

In [0]:
df_src.display()

if table not exists just bring the schema 

In [0]:
if spark.catalog.tableExists('cars_catalog.gold.dim_delaer'):
    df_sink=spark.sql('''
                      SELECT dimi_dealer_key,Dealer_Id,DealerName
                      FROM cars_catalog.gold.dim_dealer
                      ''')
else:
    df_sink=spark.sql('''
                      select 1 as dimi_dealer_key,Dealer_Id,DealerName
                      from parquet.`abfss://silver@carkrishdatalake.dfs.core.windows.net/Carsales`
                      where 1=0
                      ''')

In [0]:
df_sink.display()

##filtering new records and old records

In [0]:
df_filter=df_src.join(df_sink,df_src['Dealer_Id']== df_sink['Dealer_Id'],'left').select(df_src['Dealer_Id'],df_src['DealerName'],df_sink['dimi_dealer_key'])

In [0]:
df_filter.display()

##df_filter_old

In [0]:
df_filter_old=df_filter.filter(col('dimi_dealer_key').isNotNull())

In [0]:
df_filter.display()

###df_filter_new

In [0]:
df_filter_new = df_filter.filter(col('dimi_dealer_key').isNull()).select(df_src['Dealer_Id'],df_src['DealerName'])

In [0]:
df_filter_new.display()

#Create Surrogate Key


###Fetch the max Surrogate Key fom existing table

In [0]:
if (incremental_flag=='0'):
    max_value=1
else:
    max_value_df=spark.sql('''
                        select max(dimi_dealer_key) from cars_catalog.gold.dim_dealer ''')
    max_value=max_value_df.collect()[0][0]+1


In [0]:
df_filter_new=df_filter_new.withColumn('dimi_dealer_key',max_value+monotonically_increasing_id())

In [0]:
df_filter_new.display()

###creating final df which is equal to df_filter_old+df_filter_new

In [0]:
df_final=df_filter_new.union(df_filter_old)

#SCD TYPE_1

In [0]:
from delta.tables import DeltaTable

In [0]:

if spark.catalog.tableExists('cars_catalog.gold.dim_dealer'):
    # If the Delta table exists, perform the merge operation
    delta_table1= DeltaTable.forPath(spark, 'abfss://gold@carkrishdatalake.dfs.core.windows.net/dim_dealer')
    
    delta_table1.alias('trg').merge(df_final.alias('src'), "trg.dimi_dealer_key = src.dimi_dealer_key") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()

else:
    # If the table doesn't exist, create it by writing as a Delta table
    df_final.write.format('delta') \
        .mode('overwrite') \
        .option('path', 'abfss://gold@carkrishdatalake.dfs.core.windows.net/dim_dealer') \
        .saveAsTable('cars_catalog.gold.dim_dealer')

# Ensure the directory is formatted as Delta if needed


In [0]:
%sql
select * from cars_catalog.gold.dim_dealer